In [1]:
"""
We conduct MCMC complement sampling experiments in R^n using
random instances from MIPLIB.

Generating feasible points:
    - Too lazy to implement hitandrun, so I just randomly generate
    some vertices and then find convex combinations of them.
"""

import numpy as np
from pathlib import Path
import pickle
from pprint import pprint
from scipy.optimize import linprog

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.mixture import GaussianMixture

In [6]:
def vertex_sampler(A_mat, b_vec, randomizer=np.random.rand, n_samples=200):
    m, n = A_mat.shape
    vertices = []
    pts = []
    for _ in range(n_samples):
        c_vec = randomizer(n)
        sol = linprog(c_vec, -A_mat, -b_vec)
        if sol.success == True:
            vertices.append(np.array(sol.x))
        else:
            sol = linprog(-c_vec, -A_mat, -b_vec, method='interior-point')
            if sol.success == True:
                vertices.append(np.array(sol.x))
    vertices = np.array(vertices)
    if len(vertices) < 2:
        print('Error! Could not find enough solutions.')
        return -1
    for _ in range(n_samples):
        combo = np.random.dirichlet(np.ones(n_samples))
        pts.append(np.dot(combo, vertices))
    return pts


def load_and_generate_samples(fname, n_samples=200):
    data = pickle.load(open(fname, 'rb'))
    A_mat, b_vec = data['A_mat'], data['b_vec']
    feas_pts = vertex_sampler(A_mat, b_vec, n_samples=n_samples)

In [ ]:
def direction_sample_helper(con):
    """ Samples a random point from the unit ball then checks with
    the a vector that con'pt > 0
    """
    wrong_direction = 1
    n = len(con)
    while wrong_direction == 1:
        pt = np.random.rand(n) - 0.5
        pt = pt / np.linalg.norm(pt)
        if np.dot(con, pt) >= 0:
            wrong_direction = 0
    return pt


def direction_sample(A_mat, b_vec, bd_pt):
    """ First identifies the relevant constraint on the boundary,
    then calls sample helper."""
    ind = list(np.isclose(np.dot(A_mat, bd_pt), b_vec)).index(True)
    con = A_mat[ind]
    return direction_sample_helper(con)


def get_next_bd_pt(A_mat, b_vec, bd_pt, dir_pt):
    """ First removes boundary constraints, then finds nearest
    boundary."""
    weights = np.array([(bi - np.dot(ai, bd_pt)) / np.dot(ai, dir_pt) for ai, bi in zip(A_mat, b_vec)])
    weights[weights <= 0] = 99
    weight = min(weights)

    return bd_pt + weight * dir_pt


def shake_n_bake(A_mat, b_vec, init_pt, n_samples=10, scale=2):
    """
    1. randomly sample direction vector (r)
    2. randomly sample magnitude (xi)
    3. add infeasible point (y - xi * r, y)
    4. get next boundary point
    """
    dataset = []
    bd_pt = init_pt
    while len(dataset) < n_samples:
        r = direction_sample(A_mat, b_vec, bd_pt)
        xi = np.random.exponential(scale=scale)
        infeas_pt = bd_pt - xi * r
        dataset.append((infeas_pt, bd_pt))
        bd_pt = get_next_bd_pt(A_mat, b_vec, bd_pt, r)
    return dataset

def shake_n_bake_init(A_mat, b_vec, randomizer=np.random.rand):
    m, n = A_mat.shape
    vertices = []
    pts = []
    for i in range(10):
        c_vec = randomizer(n)
        sol = linprog(c_vec, -A_mat, -b_vec)
        if sol.success == True:
            return np.array(sol.x)
    return 'Error'
    

In [7]:
def experiment_MCMC(
        fname, 
        n_samples=200,
        randomizer=np.random.rand,
        test_size=0.2,
    ):
    data = pickle.load(open(fname, 'rb'))
    A_mat, b_vec, b_relax = data['A_mat'], data['b_vec'], data['b_relax']
    print(A_mat.shape)
    feas_pts = vertex_sampler(A_mat, b_vec, n_samples=n_samples, randomizer=randomizer)
    
    init_pt = shake_n_bake_init(A_mat, b_relax)
    dataset = shake_n_bake(A_mat, b_relax, init_pt, n_samples=n_samples, scale=5)
    infeas_pts, bd_pts = zip(*dataset)
    
    train_feas, test_feas, _, _ = train_test_split(feas_pts, np.ones(len(feas_pts)), test_size=test_size)
    X = np.vstack((train_feas, infeas_pts))
    train_feas_labels = np.array([np.ones(len(train_feas))]).T
    train_infeas_labels = np.array([np.zeros(len(infeas_pts))]).T
    y = np.vstack(
        (train_feas_labels, train_infeas_labels)
    )
    svc = SVC()
    svc.fit(X, y)
    print(svc.predict(test_feas))
    acc = svc.score(test_feas, np.ones(len(test_feas)))
    print(acc)


fname = 'miplib/gen-ip054_R1.pickle'
experiment_MCMC(fname, n_samples=50, test_size=0.8, randomizer=np.random.rand)

(57, 30)


ValueError: Phase 1 of the simplex method failed to find a feasible solution. The pseudo-objective function evaluates to 1.2e-12 which exceeds the required tolerance of 1e-12 for a solution to be considered 'close enough' to zero to be a basic solution. Consider increasing the tolerance to be greater than 1.2e-12. If this tolerance is unacceptably  large the problem may be infeasible.

In [3]:
from pathlib import Path
import pickle

# params
p = Path('miplib/')

for fname in p.iterdir():
    if fname.suffix == '.pickle':
        print(fname)
        experiment_MCMC(fname, n_samples=200)

miplib/neos16_R1.pickle


NameError: name 'experiment_MCMC' is not defined